# EDA van Schoolwebsites
Deze notebook bevat de Exploratory Data Analysis van de URLs van schoolwebsites. Hierin kan je vinden wat de formats zijn van de URLs en HTMLs van de schoolwebsites. 

## Key Points
- URLs hebben verschillende formats, hier kan je vinden welke
- HTML's bevatten \<a\> ... \</a\> anchor waarin een interne of externe link staat
- In deze anchor staat een HREF
- In deze HREF zit de link naar interne of externe site
- Deze HREF verschilt van format per school

## Kaders
*Doelen*: 
- Exploreren van URL formats
- Exploreren van Response codes van deze URls
- Exploreren van HTML's en Soup objecten van deze URls
- Exploreren van de formats van de HREFs op de schoolwebsite HTMLs zodat de scraper alle interne links correct kan afgaan op zoek naar de juiste bestanden.
- Exploreren van downloadbare bestandstypes en downloadvormen

## Risico's
- Sommige scholen hebben verkeerd begin van url: wvwv. of ww. 
- Sommige scholen hebben een url doorgegeven die naar een externe site leidt (dit is dan de daadwerkelijke site)
- Formats van de href links in de schoolwebsite htmls liggen ver uit elkaar, lastig om alles correct mee te nemen
- Sommige scholen hebben een kalender waarbij de data/maanden/jaren/etc. worden gezien als interne link, de scraper kan vast komen te zitten door alle interne links af te willen zoeken en in de kalender vast komen te zitten.
- Sommige documenten bevatten niet de volledige url van de schoolsite
- Sommige url worden doorverwezen, bijvoorbeeld naar een andere host (google)
- Sommige requests geven een 403 error: max retries

## Import Packages
Deze cell importeert benodigde packages.

In [1]:
import requests
from bs4 import BeautifulSoup
from time import sleep, strftime
import pandas as pd
import numpy as np
import datetime as dt
import os
import re
import json
import pickle
from multiprocessing import Pool
from urllib.parse import urlparse
from urllib.parse import urljoin
from urllib3.exceptions import InsecureRequestWarning

# Import functions from scraper_utils.py
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
from scraper_utils import *

## Load Data
Deze cellen laden de benodigde data.

In [2]:
# Laad txt met urls in een pandas dataframe
orgs = 'I:\Team Data-Science\Projecten\webscraper\data\org_url_20210511.txt'
orgs_df = pd.read_csv(orgs, delimiter = "\t")
urls = set(orgs_df['URL'][orgs_df['CODE_SOORT']=='BAS'].values) #<-- gebruik set() om dubbelingen te verwijderen
urls_nr = orgs_df[['NR_ADMINISTRATIE', 'URL']][orgs_df['CODE_SOORT']=='BAS'].drop_duplicates(subset='URL').values

## EDA of URLs
Exploratory Data Analysis van de url formats en stijlen.

In [10]:
# beautify urls: http://www.<domein>.nl/ of http://<domein>.nl/
b_urls = []
for url in urls:
    b_url = beautify_url(url)
    b_urls.append(b_url)
b_urls = np.array(b_urls)
b_urls

array(['http://www.talentencampusvenlo.nl', 'http://www.europaschool.nl',
       'http://www.cbsdefonteinlutten.nl', ...,
       'http://www.beatrixschool.nl', 'http://www.entree.unicoz.nl',
       'http://www.wilhelminahilversum.nl'], dtype='<U61')

In [9]:
## sommige scholen hebben verkeerde www.: ww. , wvwv.
## sommige urls zijn in caps

In [10]:
# Check welke urls geen www. bevatten
for url in urls:
    if 'www' not in url:
        print(url)

https://jls.noventa.nl
https://bsdenieuwevandale.nl
kcdebolster.nl
regenboogalmere.nl
kameleon.wereldkidz.nl
https://bsdemheyster.nl/
palet.wereldkidz.nl
laureyn.ogperspecto.nl
http://visserthooft.tabijn.nl/
https://obsmettegeupel.saamscholen.nl/
https://hetspoor.wereldkidz.nl/
tvogelnest.ogperspecto.nl
dekriekenhof.csgdewaard.nl
obshetstartblok.nl
irisschool.ogperspecto.nl
bmschoolstatia.multiply.com
rietstek.pentaprimair.nl
nautilus.quadraten.nl
basisschoolhetavontuur.nl
stjozef.ogperspecto.nl
opdreef.wereldkidz.nl
palet.groenlo.kennisnet.nl
vuurtoren.openbaaronderwijsgroningen.nl
achtbaan.wereldkidz.nl
https://devuurvlinder.info
WWW.smdbhb.nl
bsclinge.ogperspecto.nl
http://keuchenius.csgdewaard.nl
kompas.pcboapeldoorn.nl
rkbsdeklimboom-ssba.nl
alhambraa.nl
dekameleon.ogperspecto.nl
reinboge.noventa.nl
ww.vijfblad.nl
rehoboth.sco-t.nl
https://de-omnibus.com/
cbsderegenboognieuwdorp.nl
marijkeschool.ogperspecto.nl
basisskoalledetwirre.nl
deschakel.ogperspecto.nl
vrijburg.tabijn.nl
cbs

In [11]:
# check welke urls WEL http of https bevatten
for url in urls:
    if 'http' in url:
        print(url)

https://jls.noventa.nl
http://www.oranjenassau.cnsede.nl
http://www.swsdesprang.nl
https://bsdenieuwevandale.nl
https://bsdemheyster.nl/
http://visserthooft.tabijn.nl/
https://obsmettegeupel.saamscholen.nl/
https://hetspoor.wereldkidz.nl/
https://www.rkbsdehoeksteen.nl
https://devuurvlinder.info
http://keuchenius.csgdewaard.nl
https://www.opwaardz.nl/
https://de-omnibus.com/
https://www.daltonkc-helenparkhurst.nl/
https://www.daltonschooldebundeling.nl/
https://www.basisschooldeparadijsvogel.nl/
https://deontdekkingsreis.wereldkidz.nl/
https://wilhelmina.rehoboth.nu/
http://www.despringplank-eindhoven.nl
http://helmgras.tabijn.nl
https://kompas.quadraten.nl/
https://jozefschoolschipluiden.wsko.nl
http://defontein.sco-t.nl
https://adeborg.quadraten.nl
https://oosterhoogebrugschool.o2g2.nl
https://steijaertschool.nl/
https://www.springplank-hd.nl/
http://www.sco-t.nl/websites/detweeklank/school/
https://www.driesprong.eu/
http://stichtingkoc.nl/leerunit
https://deploeg.o2g2.nl
http://www

In [12]:
# Verboden tekens
for url in urls:
    if ',' in url:
        print(url)

www.ikcdriemaster,nl
www.cbsdeontdekking,nl


In [12]:
# Print mogelijke domeinstijlen voor alle urls
for url in urls:
    print(domain_styles(url))

p://www.meestergijbels.nl/', 'http://www.meestergijbels.nl', 'https://www.meestergijbels.nl/', 'https://www.meestergijbels.nl', 'http://meestergijbels.nl/', 'http://meestergijbels.nl', 'https://meestergijbels.nl/', 'https://meestergijbels.nl'])
(['driepas.nl/', 'driepas.nl', 'www.driepas.nl/', 'www.driepas.nl'], ['http://www.driepas.nl/', 'http://www.driepas.nl', 'https://www.driepas.nl/', 'https://www.driepas.nl', 'http://driepas.nl/', 'http://driepas.nl', 'https://driepas.nl/', 'https://driepas.nl'])
(['dewilge.nl/', 'dewilge.nl', 'www.dewilge.nl/', 'www.dewilge.nl'], ['http://www.dewilge.nl/', 'http://www.dewilge.nl', 'https://www.dewilge.nl/', 'https://www.dewilge.nl', 'http://dewilge.nl/', 'http://dewilge.nl', 'https://dewilge.nl/', 'https://dewilge.nl'])
(['bslahrhof-sittard.nl/', 'bslahrhof-sittard.nl', 'www.bslahrhof-sittard.nl/', 'www.bslahrhof-sittard.nl'], ['http://www.bslahrhof-sittard.nl/', 'http://www.bslahrhof-sittard.nl', 'https://www.bslahrhof-sittard.nl/', 'https://ww

In [60]:
# Check welke urls niet geopend kunnen worden met requests package
# Deze cell runt in ongeveer 2.5 uur
count = 0
exceptions = set()
for b_url in b_urls:
    try:
        html = requests.get(b_url).text
    except:
        exceptions.add(b_url)
        continue
    count+=1
print(exceptions)

{'http://www.opdehoeksteen.nl/', 'http://www.annefrank-tabijn.nl/', 'http://www.clausschool.nl/', 'http://www.nicobulderschool.nl/', 'http://www.immanuelschool-nunspeet.net/', 'http://www.bsdegoedeherder.nl/', 'http://www.gbs-deschakel.nl/', 'http://www.het-tangram.nl/', 'http://www.prinsconstantijngoor.nl/', 'http://www.theothijssen.nl/', 'http://www.obsheteiland.nl/', 'http://www.sintgertrudis.nl/', 'http://www.marcoen.nl/', 'http://www.de-smithoek.nl/', 'http://www.wilhelminaschool-rijssen.nl/', 'http://www.denhaag.bscosmicus.nl/', 'http://www.hetoctaafkrimpen.nl/', 'http://www.obsdedelta.eu/', 'http://www.otterkolken-tabijn.nl/', 'http://www.rkbsdereiger.nl./', 'http://www.wegwijzersteenwijk.nl/', 'http://www.bsdekeerkring.nl/', 'http://www.tkofschip.obase.nl/', 'http://www.obsdegouw.nl/', 'http://www.basisschooldeevenaar.nl/', 'http://www.omnibus-akkoord-po.nl/', 'http://www.springplank.pcsl.nl/', 'http://www.bs-opdehorst.nl/', 'http://www.kwschool.net/', 'http://www.basisschool-w

In [61]:
with open("exceptions.txt", "w") as f:
    f.write(str(exceptions))

In [62]:
len(exceptions)

95

## EDA of HTML contents
Exploratory Data Analysis van de HTML formats en de bijbehorende interne links en HREFs in de \<a\> anchors.

In [59]:
# Kijk hoe een soup object eruit ziet
url = 'www.dewilge.nl/'
b_url = beautify_url(url)
html = requests.get(b_url).text
soup = BeautifulSoup(html).findAll('a')
soup

[<a class="logo" href="/">
 <img alt="De Wilge" src="https://cdn.atscholen.nl/www.dewilge.nl/template/img/logo.svg"/>
 </a>,
 <a class="btn" href="/aanmelden">AANMELDEN NIEUWE LEERLING</a>,
 <a href="/zoeken"><span class="icon-search"></span></a>,
 <a class="nav-link icon-mail" data-content="administratie@dewilge.nl" data-placement="bottom" data-toggle="popover" data-trigger="hover" href="mailto:administratie@dewilge.nl"></a>,
 <a 6854074="" class="nav-link icon-phone" data-content="Bel: 035 6854074" data-placement="bottom" data-toggle="popover" data-trigger="hover" href="tel:035"></a>,
 <a class="nav-link icon-marker" data-content="Cornelis Drebbelstraat 64 1222 SC Hilversum" data-placement="bottom" data-toggle="popover" data-trigger="hover" href="https://www.google.com/maps/place/Cornelis+Drebbelstraat+64,+1222+SC+Hilversum/data=!4m2!3m1!1s0x47c66b471cec0de1:0x813f9b51daefbb84?sa=X&amp;ved=2ahUKEwj46IPcg-bsAhWKqqQKHeWbA8AQ8gEwAHoECAwQAQ" target="_blank"></a>,
 <a class="nav-link icon

In [58]:
# Kijk hoe hrefs eruit zien
for link in soup:
    if "href" in link.attrs:
        new_page = link.attrs["href"]
        print(new_page)

/
/aanmelden
/zoeken
mailto:administratie@dewilge.nl
tel:035
https://www.google.com/maps/place/Cornelis+Drebbelstraat+64,+1222+SC+Hilversum/data=!4m2!3m1!1s0x47c66b471cec0de1:0x813f9b51daefbb84?sa=X&ved=2ahUKEwj46IPcg-bsAhWKqqQKHeWbA8AQ8gEwAHoECAwQAQ
http://ouders.parnassys.net
http://portal.atscholen.nl
/
/contact
/praktijk
/praktijk/continurooster
/praktijk/vakantierooster
/praktijk/schoolgids-de-wilge
/praktijk/naschoolse-opvang
/praktijk/belangrijke-adressen
/praktijk/jaarkalender
/onderwijs
/onderwijs/tweede-taalverwerving
/onderwijs/ipc-international-primary-curriculum
/onderwijs/ict-en-de-wilge
/onderwijs/keep-moving-bewegingsonderwijs
/onderwijs/kunst-cultuur
/onderwijs/bibliotheek-op-school
/onderwijs/begeleiding
/onderwijs/persoonlijke-en-sociale-ontwikkeling
/over-school
/over-school/team-klassenbezetting-en-plattegrond
/over-school/virtuele-rondleiding
/over-school/bestuur
/nieuws
/contacten-ouders-en-school
/contacten-ouders-en-school/ouderraad
/contacten-ouders-en-school/

In [ ]:
## Veel voorkomende hrefs in een <a> anchor (link)
# /<href>                                               <- (meest voorkomende href format), b_url + href
# <url>/<href>                                          <- beautify, en gebruik
# <b_url>/<href>                                        <- gebruik
# /<url>                                                <- remove / en gebruik
# #<iets>                                               <- / + href gebruik
# //<url>                                               <- remove //

## Volgorde
# [.index, facebook, instagram, linkedin, google, .jpg, .png, :, #] <- skip
# mailto:, javascript: (iets met : tenzij http of https)<- skip
# <iets>.jpg, <iets>.png (ongwenste file formats)       <- skip
# .index<iets>                                          <- skip
# #                                                     <- skip
# /                                                     <- skip

# MAAK EXCEPTION VOOR .pdf/.doc/.docx BESTANDEN, DEZE WORDEN NU SOMS GESKIPT

In [ ]:
# Als s uit styles in href dan http + href 
# Als s uit b_styles dan gelijk werkbaar
# Als /href dan b_url + href
# Als href_noslash dan b_rul + / + href

In [51]:
styles, b_styles = domain_styles(url)
check = styles + b_styles
check

['eltheto.sco-t.nl/',
 'eltheto.sco-t.nl',
 'www.eltheto.sco-t.nl/',
 'www.eltheto.sco-t.nl',
 'http://www.eltheto.sco-t.nl/',
 'http://www.eltheto.sco-t.nl',
 'https://www.eltheto.sco-t.nl/',
 'https://www.eltheto.sco-t.nl',
 'http://eltheto.sco-t.nl/',
 'http://eltheto.sco-t.nl',
 'https://eltheto.sco-t.nl/',
 'https://eltheto.sco-t.nl']

In [20]:
domain_styles(url)

(['eltheto.sco-t.nl/',
  'eltheto.sco-t.nl',
  'www.eltheto.sco-t.nl/',
  'www.eltheto.sco-t.nl'],
 ['http://www.eltheto.sco-t.nl/',
  'http://www.eltheto.sco-t.nl',
  'https://www.eltheto.sco-t.nl/',
  'https://www.eltheto.sco-t.nl',
  'http://eltheto.sco-t.nl/',
  'http://eltheto.sco-t.nl',
  'https://eltheto.sco-t.nl/',
  'https://eltheto.sco-t.nl'])

## EDA of Exceptions
Er zijn enkele URLs die niet ingelezen kunnen worden om welke reden dan ook, deze staan opgeslagen in de exceptions.txt file. In deze cellen wordt onderzocht waarom deze niet kunnen worden geopgend.

In [54]:
# Laad exceptions.txt in
with open('exceptions.txt', 'r') as f:
    exceptions = set(line.strip() for line in f)

In [57]:
# Print exception types
for url in exceptions:
    try:
        requests.get(url)
    except Exception as e:
        print(e)

ionPool(host='www.bsdetweesprong.nl', port=443): Max retries exceeded with url: / (Caused by ProxyError('Cannot connect to proxy.', OSError('Tunnel connection failed: 502 Tunnel Connection Failed')))
HTTPSConnectionPool(host='deoosterburcht.obase.nl', port=443): Max retries exceeded with url: / (Caused by SSLError(SSLCertVerificationError(1, '[SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: unable to get local issuer certificate (_ssl.c:1123)')))
HTTPSConnectionPool(host='www.gbs-deschakel.nl', port=443): Max retries exceeded with url: / (Caused by SSLError(SSLCertVerificationError(1, '[SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: unable to get local issuer certificate (_ssl.c:1123)')))
HTTPSConnectionPool(host='www.de-tandem.nl', port=443): Max retries exceeded with url: / (Caused by SSLError(SSLCertVerificationError("hostname 'www.de-tandem.nl' doesn't match 'de-tandem.nl'")))
HTTPSConnectionPool(host='www.depionier.nl', port=443): Max retries exceeded with u

## EDA of Doc Locations and Exceptions
Hier staat de exploratory data analysis van de web locaties van de schoolgidsen en andere documenten en van de weblocaties die niet aangeroepen konden worden door de scraper.

In [15]:
with open('docs.pkl', 'rb') as f:
    docs = pickle.load(f)

In [10]:
with open('scraper_exceptions.pkl', 'rb') as f:
    exceptions = pickle.load(f)

In [12]:
len(exceptions)

540

In [11]:
# Proportie van de gescrapete documentlocaties die daadwerkelijk een schoolgids (string) lijken te hebben
empty = 0
sg_counter = 0
not_seg_counter = 0
for key in docs:
    rec = docs[key]
    if not rec:
        empty += 1
    else:
        sg_check = False
        for doc in rec:
            if 'sg' in doc.lower() or 'gids' in doc.lower():
                sg_check = True
        if sg_check:
            sg_counter += 1
        elif not sg_check:
            not_seg_counter += 1

print(f'Amount of urls checked: {len(urls)}')
print(f'Amount of unreached urls: {len(exceptions)}')
print(f'Amount of urls with no documents: {empty}')
print(f'Amount of urls with no schoolgids: {not_seg_counter}')
print(f'Amount of urls with schoolgids: {sg_counter}')

Amount of urls checked: 5370
Amount of unreached urls: 540
Amount of urls with no documents: 2990
Amount of urls with no schoolgids: 920
Amount of urls with schoolgids: 918


In [20]:
540 + 2990 + 920 + 918

5368

In [23]:
# Als url wel documenten heeft maar geen gids: wat dan wel?
for key in docs:
    rec = docs[key]
    if rec:
        check = False
        for doc in rec:
            if 'sg' in doc.lower() or 'gids' in doc.lower():
                check = True
        if not check:
            print(key, docs[key])

0Kindplein%20't%20Centrum%2003-09-2020.pdf?ver=2020-10-01-161711-203", '/Portals/1403/docs/weekbrief%201%2021-08-2020.pdf?ver=2020-08-24-131605-333', "/Portals/1403/docs/Weekbrief%20Kindplein%20't%20Centrum%2001-10-2020.pdf?ver=2020-10-01-161711-430"}
http://www.domheldercamaraschool.nl/ {'https://domheldercamaraschool.nl/wp-content/uploads/2021/06/Nieuwsbrief-1-juni-2021.pdf', 'https://domheldercamaraschool.nl/wp-content/uploads/2021/04/Nieuwsbrief-13-april-2021.pdf', 'https://domheldercamaraschool.nl/wp-content/uploads/2021/05/Nieuwsbrief-11-mei-2021.pdf'}
http://www.rijnlandslyceum.nl/ {'https://rijnlandslyceum.nl/sites/default/files/files/privacy%20policy%20SRL%20def%20ENG%20DEF.pdf', 'https://rijnlandslyceum.nl/sites/default/files/files/Privacy%20Policy%20SRL%20versie%20internet%20DEF.pdf'}
http://www.klinkerschiedam.nl/ {'https://jeugdfondssportencultuur.nl/wp-content/uploads/2019/07/Ouderkaart-2019.pdf'}
http://www.bs-hobbitstee.nl/ {'/bestanden/508392/Aanmeldformulier-de-Hobbit

## EDA for Multiprocessing Scraper
Deze cellen testen de multiprocessing module voor de scraper.

In [12]:
urls_test = []
counter = 0
for key in docs:
    urls_test.append(key)
    counter += 1
    if counter > 4: break

In [13]:
urls_test

['http://www.obsdesteven.nl/',
 'http://www.ksu-stdominicus.nl/',
 'http://www.hofdijckschool.nl/',
 'http://www.detoermalijn.com/',
 'http://www.kristalla.nu/']

In [ ]:
from multiprocessing import Pool
import workers
if __name__ ==  '__main__': 
 num_processors = 3
 p=Pool(processes = num_processors)
 output = p.map(workers.worker,[i for i in range(0,3)])
 print(output)

In [14]:
%%timeit
from scraper import index_url
if __name__ ==  '__main__': 
    with Pool(2) as p:
        p.map(index_url, urls_test)

26.1 s ± 1.17 s per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [18]:
%%timeit
from scraper import index_url
if __name__ ==  '__main__': 
    for url in urls_test:
        index_url(url)

The slowest run took 4.07 times longer than the fastest. This could mean that an intermediate result is being cached.
18 µs ± 13.1 µs per loop (mean ± std. dev. of 7 runs, 1 loop each)


Lijkt er niet op dat multiprocessing veel gaat uitmaken in de scraper.

## EDA for Empty Return Values of Scraper
Deze cellen bevatten de exploratory data analysis van de urls waar geen documenten worden gevonden door de scraper.

In [11]:
with open('docs.pkl', 'rb') as f:
    docs = pickle.load(f)

In [12]:
empty = []
for key in docs:
    if not docs[key]:
        empty.append(key)

In [3]:
for nr, url in urls_nr:
    if nr == '00GV':
        print(url)

www.mariavs.nl


In [27]:
requests.packages.urllib3.disable_warnings(category=InsecureRequestWarning)
b_url = beautify_url('www.montessorischooldekraal.nl')
b_url= requests.get(b_url, allow_redirects=True, verify=False).url
b_url

'https://www.montessorischooldekraal.nl/'

In [22]:
def index_url(url, max_depth = 2):
  # Deze functie indexeert de hele website gegeven een URL (map van de hele website en interne links), tot een maximale diepte
  depth = 0
  checked = set()
  b_url = beautify_url(url)
  urls = set([b_url])

  while depth <= max_depth:
    temp = urls.copy()
    for url in temp:
      if url not in checked:
        checked.add(url)
        # kijk of link intern is of niet (anders niet get_links)
        links = get_links(url)
        for link in links:
          if check_href(link) and is_internal(b_url, link):
            urls.add(link)
          else:
            urls.add(link)
            checked.add(link)
    depth += 1
  urls.update(checked)
  return urls

In [21]:
def check_href(href):
  # Deze functie checkt of een href werkbaar is voordat de href aan een url parser wordt gegeven

  # Check verboden woorden en tekens en file formats
  skip = ['.index', 'facebook', 'instagram', 'linkedin', 'twitter', '.jpg', '.png']
  if not any(s in href for s in skip) and href!='/' and href!='#' and href!='':
    return_val = True
    # Check :, corrigeer voor http
    if 'http' in href:
      if len(re.findall(':', href))>1:
        return_val = False
      else:
        return_val = True
    else:
      if len(re.findall(':', href))>0:
        return_val = False
      else:
        return_val = True
  else:
    return_val = False
  return return_val

In [5]:
index = index_url('https://agnietenschool.nl/', max_depth=2)

In [6]:
index

{'http://www.agnietenschool.nl',
 'http://www.school-design.nl',
 'http://www.zeeluwe.nl',
 'https://agnietenschool.nl',
 'https://agnietenschool.nl/',
 'https://agnietenschool.nl/?page_id=2527',
 'https://agnietenschool.nl/?page_id=2673',
 'https://agnietenschool.nl/?page_id=2730',
 'https://agnietenschool.nl/?page_id=2752',
 'https://agnietenschool.nl/?page_id=2760',
 'https://agnietenschool.nl/?page_id=2793',
 'https://agnietenschool.nl/?page_id=2801',
 'https://agnietenschool.nl/?page_id=2839',
 'https://agnietenschool.nl/?page_id=2849',
 'https://agnietenschool.nl/?page_id=2862',
 'https://agnietenschool.nl/?page_id=2869',
 'https://agnietenschool.nl/?page_id=2881',
 'https://agnietenschool.nl/?page_id=2909',
 'https://agnietenschool.nl/?page_id=2919',
 'https://agnietenschool.nl/?page_id=2927',
 'https://agnietenschool.nl/?page_id=2934',
 'https://agnietenschool.nl/?page_id=2947',
 'https://agnietenschool.nl/?page_id=2954',
 'https://agnietenschool.nl/?page_id=2971',
 'https://ag

In [7]:
download_docs(index)

InvalidURL: Failed to parse: http://tel:0525-684400

In [8]:
is_downloadstream('tel:0525-684400')

InvalidURL: Failed to parse: http://tel:0525-684400

In [25]:
requests.packages.urllib3.disable_warnings(category=InsecureRequestWarning)
b_url = beautify_url('https://www.mariavs.nl/starnet/modules/sn_downloads/download.php?dwnl_id=301&cat_id=13')
headers = set_headers()
head = requests.head(b_url, allow_redirects=True, verify=False, headers=headers).headers
head.get('Content-Disposition', '').split('filename')[1].replace('=', '').replace('"', '')

'19-20_Informatiegids.pdf'

In [42]:
for i in index:
    if '.pdf' in i:
        print(i)

https://www.hetsterrenlicht.nl/Media/view/22018/Schoolgids+2020-2021.pdf
https://www.hetsterrenlicht.nl/Media/download/20546/Verzoek+om+extra+verlof.pdf
https://www.hetsterrenlicht.nl/Media/view/21019/Schoolondersteuningsprofiel+2018_1.pdf


In [46]:
download_docs(index)

In [52]:
headers = set_headers()
allow_redirects = True
verify = False
requests.packages.urllib3.disable_warnings(category=InsecureRequestWarning) # Verify=False geeft een insecure request warning
html = requests.get('https://drive.google.com/drive/folders/0B8gWhbjK3a0ES3lnbU1RLTY3QnM?resourcekey=0-gfp6z66iCbRn8iwH4gT9gw', headers=headers, verify=verify,allow_redirects=allow_redirects).text
soup = BeautifulSoup(html, features='lxml').findAll('a')            
for link in soup:
    #print(link)
    if 'href' in link.attrs:
        href = link.attrs['href']
        print(href)

https://support.google.com/drive/answer/2375082
https://drive.google.com/?tab=oo
https://www.google.nl/intl/nl/about/products?tab=oh
https://accounts.google.com/ServiceLogin?hl=nl&passive=true&continue=https://drive.google.com/drive/folders/0B8gWhbjK3a0ES3lnbU1RLTY3QnM%3Fresourcekey%3D0-gfp6z66iCbRn8iwH4gT9gw&service=writely&ec=GAZAMQ
https://drive.google.com/?tab=oo


## EDA of HTTP Response Codes
Deze cellen bevatten de exploratory data anlysis van de response codes die de requests package terugkrijgt bij het ophalen van de html van de school urls.

In [20]:
headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.77 Safari/537.36 Edg/91.0.864.41'}
exceptions = set()
for nr, url in urls_nr:
    b_url = beautify_url(url)
    try:
        html = requests.get(b_url, headers=headers)
        if html.status_code != 200:
            print(f'{nr} {b_url} response: {html.status_code}')
    except:
        exceptions.add(url)

# 296 urls die een niet 200 response code geven

http://www.planthof.nl/ response: 504
http://www.onderwijsprimair.nl/obs-de-hobbitstee/ response: 404
http://www.borgh.quadraten.nl/ response: 504
http://www.deschakelbest.nl/ response: 502
http://www. sterrenpad.fortior.nl/ response: 504
http://www.remmeltbooy.quadraten.nl/ response: 504
http://dekiem.new.verdel.it/ response: 502
http://ww.klosterhufke.nl/ response: 504
http://www.paradjisvogel.nl/ response: 504
http://www.branding-tabijn.nl/ response: 504
http://www.bsdekemp.nl/ response: 502
http://www.fransiscusschool.nl/ response: 504
http://www.rbsdefontein.nl/ response: 504
http://www.nijegaast.nl/barte/ response: 404
http://www.horizon.h3o.nl/ response: 504
http://www.rank-pcbodongeradeel.nl/ response: 403
http://wwww.kindcentrumdeklimboom.nl/ response: 504
http://www.cbsdewegwijzeropperdoes.nl/ response: 504
http://www.spcol.nl/ response: 504
http://www.nijegaast.nl/fluessen/ response: 404
http://www.wilhelminabasisschool.nl/ response: 403
http://www.binnenplein.nl/ response: 

In [21]:
len(exceptions)

86

In [27]:
headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.77 Safari/537.36 Edg/91.0.864.41'}
url = 'http://www.sparklingkids.nu'
b_url = beautify_url(url)
html = requests.get(b_url, headers=headers, allow_redirects = True)
print(html.status_code)

504


## EDA of Sitemaps
Deze cellen bevatten de code om te bekijken of all urls een sitemap.xml bevatten, dit vergemakkelijkt het proces om te zoeken op een website naar de documenten.
- Bijna geen websites met sitemap.xml

In [53]:
headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.77 Safari/537.36 Edg/91.0.864.41'}
no_sitemap = []
for nr, url in urls_nr:
    b_url = beautify_url(url)
    try:
        requests.get(urljoin(url,'sitemap.xml'), headers=headers)
    except:
        print(f'{nr} {url}')
        no_sitemap.append(b_url)
        continue

oss.nl
20KT www.deblinkerd-oss.nl
20KW www.obsdeplevier.nl
20OQ www.obskortland.nl
20OW www.obseinder.nl
20PN www.degroeiplaneet.nl
20SC www.holendrechtschool.nl
20SE www.huizingaschool.nl
20SG www.kinkerbuurtschool.nl
20SK www.louisbouwmeesterschool.nl
20SL www.obsdetoekomst.nl
20SZ bsdeschatgraver.nl
20TD www.opgenhei.nl
20TN www.valder.nl
20TP www.obsdester.nl
20TQ www.rosaboekdrukker.nl
20TR www.merkelbachschool.nl
20TT www.winterkoninkje.nl
20TV www.depinksterbloem.nl
20TW www.5mw.nl
20TX www.annefrank-montessori.nl
20UL www.t-klankbord.nl
20UO www.atbdespringplank.nl
20UP www.7emontessori.nl
20UU www.marberna.nl
20UV www.10emeidoorn.nl
20UW www.tweemasterleiden.nl
20VA www.14e-montessori.nl
20VB www.josephschool.nl
20VD www.pacellischool.nl
20VF www.singelleiden.nl
20VG www.narcisquerido.nl
20VJ www.nicolaasmaes.nl
20VK www.witteolifant.nl
20VN www.troelstraschool.nl
20VP www.amgs.net
20VU www.dewaterkant.net
20VX www.multatulischool.nl
20XA www.oscarcarre.nl
20XO www.bredeschool

In [54]:
len(no_sitemap)

5316

## EDA of Succes Rate of Scraper
Deze cellen bevatten de Exploratory Data Analysis van de success rate van de scraper om te kijken hoeveel van de directories worden gevuld. Daarnaast kijken we hoeveel van de directories worden gevuld met een document dat voldoet aan bepaalde keywords.

In [34]:
path = 'C:\\Users\\ad010sup\\Documents\\'
folders = os.listdir(path)
count_empty = 0
for f in folders:
    if f in urls_nr:
        if not os.listdir(path+f):
            count_empty += 1

In [35]:
success_rate = 1 - count_empty/len(folders)
print(success_rate)

0.6879581151832461


In [ ]:
if not any(s in href for s in skip)

In [38]:
path = 'C:\\Users\\ad010sup\\Documents\\'
folders = os.listdir(path)
keywords = ['sg', 'gids', 'schoolgids']
#houdt rekening met lower()
hits = 0
folder_count = 0
for f in folders:
    if f in urls_nr:
        folder_count += 1
        subfolders = os.listdir(path+f)
        if subfolders:
            for sf in subfolders:
                if any(kw in sf.lower() for kw in keywords):
                    hits += 1

In [39]:
success_rate = hits/len(folders)
print(success_rate)

0.7633507853403141
